In [3]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import os
import pickle
import numpy as np
import itertools
import pandas as pd
from collections import Counter
from scipy.sparse import csr_matrix
import scipy.sparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score# Поменяйте на свой путь к данным

In [41]:
# Поменяйте на свой путь к данным
# PATH_TO_DATA = '/tmp/working/machinlearning/6/ident/kaggle_data/'
PATH_TO_DATA = '/tmp/working/ml/machinlearning/6/ident/kaggle_data/'

In [42]:
sites = ["site{}".format(i) for i in range(1,11)]
times = ['time{0}'.format(i) for i in range(1,11)]   

new_feat=['session_timespan', '#unique_sites', 'start_hour', 
                 'day_of_week',]

In [43]:
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
                       index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
                      index_col='session_id')
with open(os.path.join(PATH_TO_DATA, 'site_dic.pkl'),'rb') as site_dic_pkl:
    site_dic = pickle.load(site_dic_pkl)

In [44]:
train_test_df = pd.concat([train_df, test_df])

In [45]:
train_test_df_sites = train_test_df[['site1', 'site2', 'site3', 
                                     'site4','site5', 
                                     'site6','site7', 'site8', 
                                     'site9', 'site10']].fillna(0).astype('int')

In [46]:
train_test_df[times]=train_test_df[times].apply(pd.to_datetime)
train_test_df[sites]=train_test_df[sites].fillna(0).astype(int)

time_df = pd.DataFrame(index=train_test_df.index)
time_df['max'] = train_test_df[times].max(axis=1,skipna=True)
time_df['min'] = train_test_df[times].min(axis=1,skipna=True)
time_df['seconds'] = (time_df['max'] - time_df['min']) / np.timedelta64(1, 's')

train_test_df['#unique_sites']=[len(np.unique(i)) for i in train_test_df[sites].values]

train_test_df['start_hour'] = time_df['min'].apply(lambda x: x.hour)
train_test_df['day_of_week'] = time_df['min'].apply(lambda x: x.dayofweek)
train_test_df['session_timespan']=time_df['seconds']

In [47]:
indptr = [0]
count = 0
indices = []
data = []
for col in train_test_df_sites.values:
    counter = Counter(col)
    count += len(counter)
    indptr.append(count)
    indices += counter.keys()
    data += counter.values()

In [48]:
X_new_feat=train_test_df[new_feat]
X_new_feat_sparce = csr_matrix(X_new_feat.values)

In [49]:
train_size, test_size = train_df.shape[0],test_df.shape[0]

In [58]:
! cd $PATH_TO_DATA && pwd

/tmp/working/ml/machinlearning/6/ident/kaggle_data


In [50]:
# X_train_test_sparse = csr_matrix((data, indices, indptr), shape=(train_test_df_sites.shape[0],len(site_dic)))[:,1:]
# # X_new_feat_train_test = scipy.sparse.hstack((X_train_test_sparse,X_new_feat_sparce),format='csr')

# # X_train_sparse = X_new_feat_train_test[:train_size]
# # X_test_sparse = X_new_feat_train_test[-test_size:]
# y = train_df.user_id

In [59]:
X_train_test_sparse = csr_matrix((data, indices, indptr), shape=(train_test_df_sites.shape[0],len(site_dic)))[:,1:]
X_train_sparse = X_train_test_sparse[:train_size]
X_test_sparse = X_train_test_sparse[-test_size:]
y = train_df.user_id

In [51]:
print(X_new_feat_train_test.shape)

(229266, 36659)


In [19]:
with open(os.path.join(PATH_TO_DATA, 'X_train_sparse.pkl'), 'wb') as X_train_sparse_pklse_pkl:
    pickle.dump(X_train_sparse,X_train_sparse_pklse_pkl,protocol=2)
with open(os.path.join(PATH_TO_DATA, 'X_test_sparse.pkl'), 'wb') as X_test_sparse_pkl:
     pickle.dump(X_test_sparse,X_test_sparse_pkl,protocol=2)
with open(os.path.join(PATH_TO_DATA, 'train_target.pkl'), 'wb') as train_target_pkl:
    pickle.dump(y,train_target_pkl)

### Классфикация 

In [65]:
sgd = SGDClassifier(loss='log',random_state=7,penalty='l2',alpha=3.4043e-05,n_jobs=-1)
# sgd=SGDClassifier(loss='log',random_state=7,n_jobs=-1)

In [66]:
%%time
sgd.fit(X_train_sparse,y)

CPU times: user 2min 56s, sys: 29.1 s, total: 3min 25s
Wall time: 52.7 s


SGDClassifier(alpha=3.4043e-05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', max_iter=None, n_iter=None,
       n_jobs=-1, penalty='l2', power_t=0.5, random_state=7, shuffle=True,
       tol=None, verbose=0, warm_start=False)

In [67]:
test_pred = sgd.predict(X_test_sparse)

def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

write_to_submission_file(test_pred,"test_pred")

## Voiting

In [68]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

clf1 = SGDClassifier(loss='log',random_state=7,penalty='l2',alpha=3.4043e-05,n_jobs=-1)
clf2 = RandomForestClassifier(random_state=17,n_estimators=8,verbose=10)

eclf = VotingClassifier(estimators=[('sgd', clf1), ('rf', clf2)], voting='soft',n_jobs=1)

In [ ]:
%%time
eclf.fit(X_train_sparse,y)

building tree 1 of 8


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.3s remaining:    0.0s


building tree 2 of 8


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.2min remaining:    0.0s


building tree 3 of 8


...

building tree 5 of 8


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  3.1min remaining:    0.0s


building tree 6 of 8


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  3.9min remaining:    0.0s


building tree 7 of 8


In [37]:
test_pred = eclf.predict(X_test_sparse)

def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

write_to_submission_file(test_pred,"test_pred")

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    1.3s finished


# XGB 

In [102]:
import xgboost as xgb
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import TruncatedSVD

In [ ]:
%%time
scoring = []
for n_tree in n_trees:
    estimator = ensemble.RandomForestClassifier(n_estimators = n_tree, min_samples_split=5, random_state=1)
    score = cross_validation.cross_val_score(estimator, bioresponce_data, bioresponce_target, 
                                             scoring = 'accuracy', cv = 3)    
    scoring.append(score)
scoring = np.asmatrix(scoring)

In [93]:
X_new_feat_train_test.shape

(229266, 36659)

In [105]:
spca=TruncatedSVD(n_components=1000)

In [ ]:
spca.fit(X_new_feat_train_test)